<a href="https://colab.research.google.com/github/mshojaei77/Awesome-Fine-tuning/blob/main/torchtune_examples.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch torchvision torchao
!pip install torchtune

## Training with LoRA

Works! Set `packed=True` (under `dataset`, set `max_seq_len` under `tokenizer`) and `compile=True` (under `# Training`) and it trains in 4 minutes instead of 50!

In [ ]:
import yaml

yaml_string = """
output_dir: /tmp/torchtune/qwen2_0_5B/lora_single_device # /tmp may be deleted by your system. Change it to your preference.

# Model Arguments
model:
  _component_: torchtune.models.qwen2.lora_qwen2_0_5b
  lora_attn_modules: ['q_proj', 'v_proj', 'output_proj']
  apply_lora_to_mlp: False
  lora_rank: 4  # higher increases accuracy and memory
  lora_alpha: 8  # usually alpha=2*rank
  lora_dropout: 0.0

tokenizer:
  _component_: torchtune.models.qwen2.qwen2_tokenizer
  path: /tmp/Qwen2-0.5B-Instruct/vocab.json
  merges_file: /tmp/Qwen2-0.5B-Instruct/merges.txt
  max_seq_len: 8192

checkpointer:
  _component_: torchtune.training.FullModelHFCheckpointer
  checkpoint_dir: /tmp/Qwen2-0.5B-Instruct
  checkpoint_files: [
    model.safetensors
  ]
  recipe_checkpoint: null
  output_dir: ${output_dir}
  model_type: QWEN2

resume_from_checkpoint: False

# Dataset and Sampler
dataset:
  _component_: torchtune.datasets.alpaca_cleaned_dataset
  packed: True  # True increases speed
seed: null
shuffle: True
batch_size: 4

# Optimizer and Scheduler
optimizer:
  _component_: torch.optim.AdamW
  fused: True
  weight_decay: 0.01
  lr: 2e-3

lr_scheduler:
  _component_: torchtune.training.lr_schedulers.get_cosine_schedule_with_warmup
  num_warmup_steps: 100

loss:
  _component_: torchtune.modules.loss.CEWithChunkedOutputLoss

# Training
epochs: 1
max_steps_per_epoch: null
gradient_accumulation_steps: 8  # Use to increase effective batch size
clip_grad_norm: null
compile: True  # torch.compile the model + loss, True increases speed + decreases memory

# Logging
metric_logger:
  _component_: torchtune.training.metric_logging.DiskLogger
  log_dir: ${output_dir}/logs
log_every_n_steps: 1
log_peak_memory_stats: True

# Environment
device: cuda
dtype: bf16

# Activations Memory
enable_activation_checkpointing: True  # True reduces memory
enable_activation_offloading: False  # True reduces memory

# Show case the usage of pytorch profiler
# Set enabled to False as it's only needed for debugging training
profiler:
  _component_: torchtune.training.setup_torch_profiler
  enabled: False

  #Output directory of trace artifacts
  output_dir: ${output_dir}/profiling_outputs

  #`torch.profiler.ProfilerActivity` types to trace
  cpu: True
  cuda: True

  #trace options passed to `torch.profiler.profile`
  profile_memory: False
  with_stack: False
  record_shapes: True
  with_flops: False

  # `torch.profiler.schedule` options:
  # wait_steps -> wait, warmup_steps -> warmup, active_steps -> active, num_cycles -> repeat
  wait_steps: 5
  warmup_steps: 5
  active_steps: 2
  num_cycles: 1
"""


# Convert the YAML string to a Python dictionary
yaml_dict = yaml.safe_load(yaml_string)

# Specify your file path
file_path = 'torchtune_test.yaml'

# Write the YAML file
with open(file_path, 'w') as file:
    yaml.dump(yaml_dict, file)

In [ ]:
!tune download Qwen/Qwen2-0.5B-Instruct --output-dir /tmp/Qwen2-0.5B-Instruct

Ignoring files matching the following patterns: None
.gitattributes: 100% 1.52k/1.52k [00:00<00:00, 8.04MB/s]
LICENSE: 100% 11.3k/11.3k [00:00<00:00, 49.0MB/s]
README.md: 100% 3.56k/3.56k [00:00<00:00, 26.4MB/s]
config.json: 100% 659/659 [00:00<00:00, 4.77MB/s]
generation_config.json: 100% 242/242 [00:00<00:00, 1.99MB/s]
merges.txt: 100% 1.67M/1.67M [00:00<00:00, 1.95MB/s]
model.safetensors: 100% 988M/988M [00:03<00:00, 296MB/s]
tokenizer.json: 100% 7.03M/7.03M [00:01<00:00, 5.49MB/s]
tokenizer_config.json: 100% 1.29k/1.29k [00:00<00:00, 11.7MB/s]
vocab.json: 100% 2.78M/2.78M [00:00<00:00, 11.8MB/s]
Successfully downloaded model repo and wrote to the following locations:
/tmp/Qwen2-0.5B-Instruct/.gitattributes
/tmp/Qwen2-0.5B-Instruct/config.json
/tmp/Qwen2-0.5B-Instruct/tokenizer_config.json
/tmp/Qwen2-0.5B-Instruct/model.safetensors
/tmp/Qwen2-0.5B-Instruct/merges.txt
/tmp/Qwen2-0.5B-Instruct/original_repo_id.json
/tmp/Qwen2-0.5B-Instruct/vocab.json
/tmp/Qwen2-0.5B-Instruct/generatio

In [ ]:
!tune run lora_finetune_single_device --config ./torchtune_test.yaml

INFO:torchtune.utils._logging:Running LoRAFinetuneRecipeSingleDevice with resolved config:

batch_size: 4
checkpointer:
  _component_: torchtune.training.FullModelHFCheckpointer
  checkpoint_dir: /tmp/Qwen2-0.5B-Instruct
  checkpoint_files:
  - model.safetensors
  model_type: QWEN2
  output_dir: /tmp/torchtune/qwen2_0_5B/lora_single_device
  recipe_checkpoint: null
clip_grad_norm: null
compile: true
dataset:
  _component_: torchtune.datasets.alpaca_cleaned_dataset
  packed: true
device: cuda
dtype: bf16
enable_activation_checkpointing: true
enable_activation_offloading: false
epochs: 1
gradient_accumulation_steps: 8
log_every_n_steps: 1
log_peak_memory_stats: true
loss:
  _component_: torchtune.modules.loss.CEWithChunkedOutputLoss
lr_scheduler:
  _component_: torchtune.training.lr_schedulers.get_cosine_schedule_with_warmup
  num_warmup_steps: 100
max_steps_per_epoch: null
metric_logger:
  _component_: torchtune.training.metric_logging.DiskLogger
  log_dir: /tmp/torchtune/qwen2_0_5B/lor

## Training with rsLoRA

Doesn't train---they don't support rsLoRA.

```
TypeError: lora_qwen2_0_5b() got an unexpected keyword argument 'use_rslora'
```

```python
def lora_qwen2_0_5b(
    lora_attn_modules: List[LORA_ATTN_MODULES],
    apply_lora_to_mlp: bool = False,
    lora_rank: int = 8,
    lora_alpha: float = 16,
    lora_dropout: float = 0.0,
    use_dora: bool = False,
    quantize_base: bool = False,
)
```

In [ ]:
import yaml

yaml_string = """
output_dir: /tmp/torchtune/qwen2_0_5B/lora_single_device # /tmp may be deleted by your system. Change it to your preference.

# Model Arguments
model:
  _component_: torchtune.models.qwen2.lora_qwen2_0_5b
  lora_attn_modules: ['q_proj', 'v_proj', 'output_proj']
  apply_lora_to_mlp: False
  lora_rank: 4  # higher increases accuracy and memory
  lora_alpha: 8  # usually alpha=2*rank
  lora_dropout: 0.0
  use_rslora: True

tokenizer:
  _component_: torchtune.models.qwen2.qwen2_tokenizer
  path: /tmp/Qwen2-0.5B-Instruct/vocab.json
  merges_file: /tmp/Qwen2-0.5B-Instruct/merges.txt
  max_seq_len: null

checkpointer:
  _component_: torchtune.training.FullModelHFCheckpointer
  checkpoint_dir: /tmp/Qwen2-0.5B-Instruct
  checkpoint_files: [
    model.safetensors
  ]
  recipe_checkpoint: null
  output_dir: ${output_dir}
  model_type: QWEN2

resume_from_checkpoint: False

# Dataset and Sampler
dataset:
  _component_: torchtune.datasets.alpaca_cleaned_dataset
  packed: False  # True increases speed
seed: null
shuffle: True
batch_size: 4

# Optimizer and Scheduler
optimizer:
  _component_: torch.optim.AdamW
  fused: True
  weight_decay: 0.01
  lr: 2e-3

lr_scheduler:
  _component_: torchtune.training.lr_schedulers.get_cosine_schedule_with_warmup
  num_warmup_steps: 100

loss:
  _component_: torchtune.modules.loss.CEWithChunkedOutputLoss

# Training
epochs: 1
max_steps_per_epoch: null
gradient_accumulation_steps: 8  # Use to increase effective batch size
clip_grad_norm: null
compile: False  # torch.compile the model + loss, True increases speed + decreases memory

# Logging
metric_logger:
  _component_: torchtune.training.metric_logging.DiskLogger
  log_dir: ${output_dir}/logs
log_every_n_steps: 1
log_peak_memory_stats: True

# Environment
device: cuda
dtype: bf16

# Activations Memory
enable_activation_checkpointing: True  # True reduces memory
enable_activation_offloading: False  # True reduces memory

# Show case the usage of pytorch profiler
# Set enabled to False as it's only needed for debugging training
profiler:
  _component_: torchtune.training.setup_torch_profiler
  enabled: False

  #Output directory of trace artifacts
  output_dir: ${output_dir}/profiling_outputs

  #`torch.profiler.ProfilerActivity` types to trace
  cpu: True
  cuda: True

  #trace options passed to `torch.profiler.profile`
  profile_memory: False
  with_stack: False
  record_shapes: True
  with_flops: False

  # `torch.profiler.schedule` options:
  # wait_steps -> wait, warmup_steps -> warmup, active_steps -> active, num_cycles -> repeat
  wait_steps: 5
  warmup_steps: 5
  active_steps: 2
  num_cycles: 1
"""


# Convert the YAML string to a Python dictionary
yaml_dict = yaml.safe_load(yaml_string)

# Specify your file path
file_path = 'torchtune_test.yaml'

# Write the YAML file
with open(file_path, 'w') as file:
    yaml.dump(yaml_dict, file)

In [ ]:
!tune run lora_finetune_single_device --config ./torchtune_test.yaml

INFO:torchtune.utils._logging:Running LoRAFinetuneRecipeSingleDevice with resolved config:

batch_size: 4
checkpointer:
  _component_: torchtune.training.FullModelHFCheckpointer
  checkpoint_dir: /tmp/Qwen2-0.5B-Instruct
  checkpoint_files:
  - model.safetensors
  model_type: QWEN2
  output_dir: /tmp/torchtune/qwen2_0_5B/lora_single_device
  recipe_checkpoint: null
clip_grad_norm: null
compile: false
dataset:
  _component_: torchtune.datasets.alpaca_cleaned_dataset
  packed: false
device: cuda
dtype: bf16
enable_activation_checkpointing: true
enable_activation_offloading: false
epochs: 1
gradient_accumulation_steps: 8
log_every_n_steps: 1
log_peak_memory_stats: true
loss:
  _component_: torchtune.modules.loss.CEWithChunkedOutputLoss
lr_scheduler:
  _component_: torchtune.training.lr_schedulers.get_cosine_schedule_with_warmup
  num_warmup_steps: 100
max_steps_per_epoch: null
metric_logger:
  _component_: torchtune.training.metric_logging.DiskLogger
  log_dir: /tmp/torchtune/qwen2_0_5B/l

## Training with DoRA

Works but says it will take 50 minutes on an A100 which is odd.

In [ ]:
import yaml

yaml_string = """
output_dir: /tmp/torchtune/qwen2_0_5B/lora_single_device # /tmp may be deleted by your system. Change it to your preference.

# Model Arguments
model:
  _component_: torchtune.models.qwen2.lora_qwen2_0_5b
  lora_attn_modules: ['q_proj', 'v_proj', 'output_proj']
  apply_lora_to_mlp: False
  lora_rank: 4  # higher increases accuracy and memory
  lora_alpha: 8  # usually alpha=2*rank
  lora_dropout: 0.0
  use_dora: True

tokenizer:
  _component_: torchtune.models.qwen2.qwen2_tokenizer
  path: /tmp/Qwen2-0.5B-Instruct/vocab.json
  merges_file: /tmp/Qwen2-0.5B-Instruct/merges.txt
  max_seq_len: 8192

checkpointer:
  _component_: torchtune.training.FullModelHFCheckpointer
  checkpoint_dir: /tmp/Qwen2-0.5B-Instruct
  checkpoint_files: [
    model.safetensors
  ]
  recipe_checkpoint: null
  output_dir: ${output_dir}
  model_type: QWEN2

resume_from_checkpoint: False

# Dataset and Sampler
dataset:
  _component_: torchtune.datasets.alpaca_cleaned_dataset
  packed: True  # True increases speed
seed: null
shuffle: True
batch_size: 4

# Optimizer and Scheduler
optimizer:
  _component_: torch.optim.AdamW
  fused: True
  weight_decay: 0.01
  lr: 2e-3

lr_scheduler:
  _component_: torchtune.training.lr_schedulers.get_cosine_schedule_with_warmup
  num_warmup_steps: 100

loss:
  _component_: torchtune.modules.loss.CEWithChunkedOutputLoss

# Training
epochs: 1
max_steps_per_epoch: null
gradient_accumulation_steps: 8  # Use to increase effective batch size
clip_grad_norm: null
compile: True  # torch.compile the model + loss, True increases speed + decreases memory

# Logging
metric_logger:
  _component_: torchtune.training.metric_logging.DiskLogger
  log_dir: ${output_dir}/logs
log_every_n_steps: 1
log_peak_memory_stats: True

# Environment
device: cuda
dtype: bf16

# Activations Memory
enable_activation_checkpointing: True  # True reduces memory
enable_activation_offloading: False  # True reduces memory

# Show case the usage of pytorch profiler
# Set enabled to False as it's only needed for debugging training
profiler:
  _component_: torchtune.training.setup_torch_profiler
  enabled: False

  #Output directory of trace artifacts
  output_dir: ${output_dir}/profiling_outputs

  #`torch.profiler.ProfilerActivity` types to trace
  cpu: True
  cuda: True

  #trace options passed to `torch.profiler.profile`
  profile_memory: False
  with_stack: False
  record_shapes: True
  with_flops: False

  # `torch.profiler.schedule` options:
  # wait_steps -> wait, warmup_steps -> warmup, active_steps -> active, num_cycles -> repeat
  wait_steps: 5
  warmup_steps: 5
  active_steps: 2
  num_cycles: 1
"""


# Convert the YAML string to a Python dictionary
yaml_dict = yaml.safe_load(yaml_string)

# Specify your file path
file_path = 'torchtune_test.yaml'

# Write the YAML file
with open(file_path, 'w') as file:
    yaml.dump(yaml_dict, file)

In [ ]:
!tune run lora_finetune_single_device --config ./torchtune_test.yaml

INFO:torchtune.utils._logging:Running LoRAFinetuneRecipeSingleDevice with resolved config:

batch_size: 4
checkpointer:
  _component_: torchtune.training.FullModelHFCheckpointer
  checkpoint_dir: /tmp/Qwen2-0.5B-Instruct
  checkpoint_files:
  - model.safetensors
  model_type: QWEN2
  output_dir: /tmp/torchtune/qwen2_0_5B/lora_single_device
  recipe_checkpoint: null
clip_grad_norm: null
compile: true
dataset:
  _component_: torchtune.datasets.alpaca_cleaned_dataset
  packed: true
device: cuda
dtype: bf16
enable_activation_checkpointing: true
enable_activation_offloading: false
epochs: 1
gradient_accumulation_steps: 8
log_every_n_steps: 1
log_peak_memory_stats: true
loss:
  _component_: torchtune.modules.loss.CEWithChunkedOutputLoss
lr_scheduler:
  _component_: torchtune.training.lr_schedulers.get_cosine_schedule_with_warmup
  num_warmup_steps: 100
max_steps_per_epoch: null
metric_logger:
  _component_: torchtune.training.metric_logging.DiskLogger
  log_dir: /tmp/torchtune/qwen2_0_5B/lor